In [1]:
from premise import *
import bw2data
from datapackage import Package
bw2data.projects.set_current("ei39")

In [3]:
scenarios = [
    {"model": "image", "pathway": "SSP2-RCP26", "year": 2020},
    #{"model": "image", "pathway": "SSP2-Base", "year": 2022},
    #{"model": "image", "pathway": "SSP2-Base", "year": 2025},
    #{"model": "image", "pathway": "SSP2-Base", "year": 2030},
    #{"model": "image", "pathway": "SSP2-Base", "year": 2040},
    #{"model": "image", "pathway": "SSP2-RCP26", "year": 2050},
]

sps = Package("../datapackage.json")

ndb = NewDatabase(
        scenarios=scenarios,
        source_db="ecoinvent 3.9.1 cutoff",
        source_version="3.9.1",
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
        external_scenarios=[sps,],
        use_multiprocessing=False
)

premise v.(2, 0, 2)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

In [4]:
ndb.update(["external"])

Updating: external:   0%|                       | 0/1 [00:00<?, ?it/s]

Updating: external: 100%|███████████████| 1/1 [00:05<00:00,  5.05s/it]

Done!



In [5]:
ndb.write_db_to_brightway("stem test 1")

Write new database(s) to Brightway.
Running all checks...
Anomalies found: check the change report.
Database stem test 1 already exists: it will be overwritten.
Vacuuming database 


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:14


Title: Writing activities to SQLite3 database:
  Started: 03/24/2024 16:45:05
  Finished: 03/24/2024 16:45:19
  Total time elapsed: 00:00:14
  CPU %: 93.10
  Memory %: 6.92
Created database: stem test 1
Generate scenario report.
Report saved under /Users/romain/GitHub/sweet_sure-2050-switzerland/dev/export/scenario_report.
Generate change report.
Report saved under /Users/romain/GitHub/sweet_sure-2050-switzerland/dev.


In [6]:
import yaml
fp = "../configuration_file/config.yaml"
with open(fp) as f:
    pp = yaml.safe_load(f)

In [7]:
import bw2data
bw2data.projects.set_current("ei39")
def find_ds(d):
    try:
        return [
            a for a in bw2data.Database("stem test 1")
            if a["name"] == d["name"]
            and a["reference product"] == d["reference product"]
            and a["location"] == "CH"
        ][0]
    except:
        return None

FU = []
names = []
for x in pp["production pathways"].values():
    if x["ecoinvent alias"]["name"] not in names:
        FU.append({find_ds(x["ecoinvent alias"]): 1})
        names.append(x["ecoinvent alias"]["name"])

In [8]:
FU = [f for f in FU if list(f.keys())[0] is not None]
len(FU)

39

In [12]:
import bw2data
import bw2calc
import pandas as pd
bw2data.projects.set_current("ei39")
results = []
lca = bw2calc.LCA(FU[0], ('EF v3.0 EN15804', 'climate change', 'global warming potential (GWP100)'))
lca.lci(factorize=True)
lca.lcia()
act_dict, _, _ = lca.reverse_dict()
for fu in FU:
    lca.redo_lcia(fu)
    act = list(fu.keys())[0]
    print(act["name"], act["reference product"])
    idx = lca.activity_dict[act.key]
    direct = float(lca.characterized_inventory.sum(axis=0)[:, idx])
    results.append([act["name"], direct, float(lca.characterized_inventory.sum())])
    
pd.DataFrame(results, columns=["name", "direct", "total"]).to_excel("validation pp.xlsx")

electricity production, hydro, run-of-river electricity, high voltage
electricity production, hydro, reservoir, alpine region electricity, high voltage
electricity production, hydro, pumped storage electricity, high voltage
electricity production, nuclear, boiling water reactor electricity, high voltage
electricity production, nuclear, pressure water reactor electricity, high voltage
electricity production, natural gas, combined cycle power plant electricity, high voltage
electricity production, natural gas, 10MW electricity, high voltage
electricity production, at natural gas-fired combined cycle power plant, post, pipeline 200km, storage 1000m electricity, high voltage
heat and power co-generation, natural gas, 1MW electrical, lean burn electricity, high voltage
electricity production, at co-generation natural gas-fired power plant, post, pipeline 200km, storage 1000m electricity, high voltage
heat and power co-generation, natural gas, 160kW electrical, Jakobsberg electricity, low vo

In [6]:
ndb.update(["external"])

Updating: external:   0%|                       | 0/1 [00:00<?, ?it/s]

No suppliers found for market for heavy fuel oil (SPS) in CH. No market created. This may cause linking issue.


Updating: external: 100%|███████████████| 1/1 [00:04<00:00,  4.58s/it]

Done!



In [1]:
from premise import *
import bw2data
from datapackage import Package
bw2data.projects.set_current("ei39")

sps = Package("../datapackage.json")
ndb = PathwaysDataPackage(
    scenarios=[
        {"model": "image", "pathway": "SSP2-RCP26"},
    ],
    years=[2020, 2022, 2025, 2030, 2035, 2040, 2045, 2050],
    #years=[2020],
    source_db="ecoinvent 3.9.1 cutoff", # <-- name of the database in the BW2 project. Must be a string.
    source_version="3.9", # <-- version of ecoinvent. Can be "3.5", "3.6", "3.7" or "3.8". Must be a string.
    key="tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=",
    external_scenarios=[sps,]
)


premise v.(2, 0, 2)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

In [2]:
ndb.create_datapackage(
    name="image-SSP2-stem-SPS1",
    contributors=[
        {"name": "Romain",
        "email": "r_s at me.com",}
    ],
    #transformations=["trucks", "metals"]
)

`update()` will skip the following sectors: 'buses', 'cars', 'two_wheelers'.
If you want to update these sectors, please run them separately afterwards.


Updating: metals:  55%|████████▏      | 6/11 [11:54<10:51, 130.33s/it]

Activity copper-cobalt mining, artisanal - copper-cobalt ore already exists in all locations [].
Activity copper mine operation and beneficiation, sulfide ore - copper concentrate, sulfide ore already exists in all locations [].
Activity gold mine operation and refining - copper, cathode already exists in all locations [].
Activity platinum group metal mine operation, ore with high palladium content - copper, cathode already exists in all locations [].
Activity platinum group metal, extraction and refinery operations - copper, cathode already exists in all locations [].
Activity gold mine operation and refining - gold already exists in all locations [].
Activity gold production - gold already exists in all locations [].
Activity platinum group metal, extraction and refinery operations - gold already exists in all locations [].
Activity vanadium-titanomagnetite mine operation and beneficiation - iron ore concentrate already exists in all locations [].
Activity silver mine operation with

Updating: external: 100%|████████████| 11/11 [29:48<00:00, 162.64s/it]


Done!

Extracted 1 worksheets in 0.04 seconds
Remove uncertainty data.
Extracted 1 worksheets in 0.03 seconds
Remove uncertainty data.
Extracted 1 worksheets in 0.03 seconds
Remove uncertainty data.
Extracted 1 worksheets in 0.03 seconds
Remove uncertainty data.
Extracted 1 worksheets in 0.03 seconds
Remove uncertainty data.
Extracted 1 worksheets in 0.03 seconds
Remove uncertainty data.
Extracted 1 worksheets in 0.03 seconds
Remove uncertainty data.
Extracted 1 worksheets in 0.03 seconds
Remove uncertainty data.
Write new database(s) to matrix.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Matrices saved in /Users/romain/GitHub/sweet_sure-2050-switzerland/dev/pathways/inventories/image/SSP2-RCP26/2020.
Matrices saved in /Users/romain/GitHub/sweet_sure-2050-switzerland/dev/pathwa

In [7]:
from pathways import Pathways
p = Pathways(datapackage="image-SSP2-stem-SPS1/datapackage.json",)

Solver: scikits.umfpack
Invalid datapackage: Descriptor validation error: {'path': 'mapping/mapping.yaml', 'profile': 'data-resource', 'name': 'mapping', 'format': 'yaml', 'mediatype': 'text/yaml', 'encoding': 'utf-8'} is not valid under any of the given schemas at "resources/33" in descriptor and at "properties/resources/items/oneOf" in profile
Invalid datapackage: Descriptor validation error: 'data-resource' is not one of ['tabular-data-resource'] at "resources/33/profile" in descriptor and at "properties/resources/items/properties/profile/enum" in profile


In [8]:
import numpy as np
p.calculate(
    methods=[
        #'IPCC 2021 - climate change - GWP 100a, incl. H',
        'IPCC 2021 - climate change - GWP 100a, incl. H and bio CO2',
    ],
    #methods=[m for m in p.lcia_methods if "RELICS" in m],
    regions=["CH",],
    scenarios=[
        #"SSP2-Base",
        "SSP2-RCP26",
    ],
    variables=[
        "hydro_run_of_river",
        "hydro (reservoir)",
        "hydro (pumped storage)",
        "nuclear (boiling water)",
        "nuclear (pressure water)",
        "oil (peak devices)",
        "oil (DH CHP)",
        "oil (industry CHP)",
        "gas (CCGT)",
        "gas (OCGT)",
        "gas (CCGT CCS)",
        "gas (CHP DH)",
        "gas (fuel cell DH)",
        "gas (CHP industry)",
        "gas (CHP CCS industry)",
        "gas (CHP fuel cell reformer)",
        "gas (CHP single family)",
        "gas (single family CHP fuel cell)",
        "gas (multi family CHP)",
        "gas (multi family CHP fuel cell)",
        "biogas (services CHP)",
        "gas (services CHP)",
        "gas (services CHP fuel cell)",
        "waste incineration, fossil",
        "waste incineration, fossil (CHP)",
        "waste incineration, fossil (industrial)",
        "waste incineration, fossil (CCS)",
        "waste incineration, fossil (CHP, CCS)",
        "waste incineration, fossil (industrial, CCS)",
        "waste incineration, non-fossil",
        "waste incineration, non-fossil (CHP)",
        "waste incineration, non-fossil (industrial)",
        "waste incineration, non-fossil (CCS)",
        "waste incineration, non-fossil (CHP, CCS)",
        "waste incineration, non-fossil (industrial, CCS)",
        "pv (large scale)",
        "pv (industry)",
        "pv (services)",
        "pv residential (multi family)",
        "pv residential (single family)",
        "wind",
        "geothermal",
        "hydrogen fuel cell (DH CHP)",
        "hydrogen fuel cell (industrial CHP)",
        "hydrogen fuel cell (services CHP)",
        "biomass (DH)",
        "biomass (CHP)",
        "biomass (CCS, BECCS)",
        "biomass (CCS, industrial)",
        "battery (large)",
        "battery (medium)",
        "battery (industry)",
        "battery (services)",
        "battery (single family)",
        "battery (multi family)",
        "CAES",
        "import",
    ],
    years=[
        2020,
        2022,
        2030,
        2040,
        2050
    ],
    characterization=True,
    multiprocessing=False,
)
arr = p.display_results()

Calculating LCA results for image...
--- Calculating LCA results for SSP2-RCP26...
------ Calculating LCA results for 2020...
CH is not an IAM location.
CH is not an IAM location.


0% [#] 100% | ETA: 00:00:01
Total time elapsed: 00:00:00


------ Calculating LCA results for 2022...
CH is not an IAM location.
CH is not an IAM location.


0% [#] 100% | ETA: 00:00:01
Total time elapsed: 00:00:00


------ Calculating LCA results for 2030...
CH is not an IAM location.
CH is not an IAM location.


0% [#] 100% | ETA: 00:00:01
Total time elapsed: 00:00:00


------ Calculating LCA results for 2040...
CH is not an IAM location.
CH is not an IAM location.


0% [#] 100% | ETA: 00:00:01
Total time elapsed: 00:00:00


------ Calculating LCA results for 2050...
CH is not an IAM location.
CH is not an IAM location.


0% [#] 100% | ETA: 00:00:01
Total time elapsed: 00:00:00
0% [#####] 100% | ETA: 00:00:00
Total time elapsed: 00:00:09


In [9]:
p.lca_results = p.lca_results.sum(dim="act_category")

In [10]:
p.lca_results.to_netcdf("CH.nc")

In [11]:
df = p.lca_results.interp(
    year=range(2020, 2051)
).to_dataframe("value")

In [12]:
len(df)

634353

In [13]:
df = df[df["value"]!=0.0]
df = df[~df["value"].isnull()]
len(df)

221869

In [14]:
df=df.reset_index()

In [15]:
df["year"].unique()

array([2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2020,
       2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031,
       2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040])

In [16]:
from premise.geomap import Geomap
geo = Geomap("image")

In [17]:
EU = geo.iam_to_ecoinvent_location("WEU")
EU.remove("CH")

In [18]:
df.head()

,variable,year,region,location,model,scenario,impact_category,value
0,battery (multi family),2041,CH,IN-DN,image,SSP2-RCP26,"IPCC 2021 - climate change - GWP 100a, incl. H...",2.486058
1,battery (multi family),2041,CH,EG,image,SSP2-RCP26,"IPCC 2021 - climate change - GWP 100a, incl. H...",159.762974
2,battery (multi family),2041,CH,TH,image,SSP2-RCP26,"IPCC 2021 - climate change - GWP 100a, incl. H...",90387.137597
3,battery (multi family),2041,CH,CA-NS,image,SSP2-RCP26,"IPCC 2021 - climate change - GWP 100a, incl. H...",2.294140
4,battery (multi family),2041,CH,CA-AB,image,SSP2-RCP26,"IPCC 2021 - climate change - GWP 100a, incl. H...",13.811483


In [19]:
df.loc[df["location"].isin(EU), "location"] = "EU wo CH"

In [20]:
df.loc[
    (~df["location"].isin(EU))
    &(df["location"]!="CH")
    &(df["location"]!="EU wo CH")
, "location"] = "RoW"

In [21]:
df = df.groupby([
    'variable', 'year', 'region', 'model', 'scenario', 'impact_category', 'location'
]).sum().reset_index()

In [22]:
len(df)

2213

In [85]:
elec_vars = [
        "hydro_run_of_river",
        "hydro (reservoir)",
        "hydro (pumped storage)",
        "nuclear (boiling water)",
        "nuclear (pressure water)",
        "oil (peak devices)",
        "oil (DH CHP)",
        "oil (industry CHP)",
        "gas (CCGT)",
        "gas (OCGT)",
        "gas (CCGT CCS)",
        "gas (CHP DH)",
        "gas (fuel cell DH)",
        "gas (CHP industry)",
        "gas (CHP CCS industry)",
        "gas (CHP fuel cell reformer)",
        "gas (CHP single family)",
        "gas (single family CHP fuel cell)",
        "gas (multi family CHP)",
        "gas (multi family CHP fuel cell)",
        "biogas (services CHP)",
        "gas (services CHP)",
        "gas (services CHP fuel cell)",
        "waste incineration, fossil",
        "waste incineration, fossil (CHP)",
        "waste incineration, fossil (industrial)",
        "waste incineration, fossil (CCS)",
        "waste incineration, fossil (CHP, CCS)",
        "waste incineration, fossil (industrial, CCS)",
        "waste incineration, non-fossil",
        "waste incineration, non-fossil (CHP)",
        "waste incineration, non-fossil (industrial)",
        "waste incineration, non-fossil (CCS)",
        "waste incineration, non-fossil (CHP, CCS)",
        "waste incineration, non-fossil (industrial, CCS)",
        "pv (large scale)",
        "pv (industry)",
        "pv (services)",
        "pv residential (multi family)",
        "pv residential (single family)",
        "wind",
        "geothermal",
        "hydrogen fuel cell (DH CHP)",
        "hydrogen fuel cell (industrial CHP)",
        "hydrogen fuel cell (services CHP)",
        "biomass (DH)",
        "biomass (CHP)",
        "biomass (CCS, BECCS)",
        "biomass (CCS, industrial)",
        "battery (large)",
        "battery (medium)",
        "battery (industry)",
        "battery (services)",
        "battery (single family)",
        "battery (multi family)",
        "CAES",
        "import",
    ]
df["sector"] = df["variable"].map({x: "electricity" for x in elec_vars})

In [23]:
from pivottablejs import pivot_ui
from IPython.display import HTML
pivot_ui(df, outfile_path='pivottable_ch.html')

In [22]:
p.scenarios.sel(model="image", region="CH", pathway="SSP2-RCP26", year=2050).to_dataframe("x").reset_index()

,variables,pathway,region,year,model,x
0,hydro_run_of_river,SSP2-RCP26,CH,2050,image,66.13497
1,hydro (reservoir),SSP2-RCP26,CH,2050,image,69.17823
2,hydro (pumped storage),SSP2-RCP26,CH,2050,image,13.428635
3,nuclear (boiling water),SSP2-RCP26,CH,2050,image,0.0
4,nuclear (pressure water),SSP2-RCP26,CH,2050,image,0.0
5,oil (peak devices),SSP2-RCP26,CH,2050,image,0.000007
6,oil (DH CHP),SSP2-RCP26,CH,2050,image,0.0
7,oil (industry CHP),SSP2-RCP26,CH,2050,image,0.00151
8,gas (CCGT),SSP2-RCP26,CH,2050,image,0.000023
9,gas (OCGT),SSP2-RCP26,CH,2050,image,0.016025
